In [60]:
from valurap2 import commands
import imp
imp.reload(commands)

p = commands.S3GPort()
p.S3G_OUTPUT(63, 33)
p.S3G_OUTPUT(7, 0x00000000)
p.S3G_INPUT(63)

33

In [2]:
s_t = 0.001
f = 50000000
sb = 40

n = 400 * 32
t = 1

v = n * (2 ** sb) / f / t
v1 = v * 2
n_s = t / s_t
a = v1 / n_s
dt = f * s_t
v, a, dt, n_s

(281474976.710656, 562949.9534213119, 50000.0, 1000.0)

In [73]:
import time
a = [
    p.BUF_OUTPUT(0, 50000),        # step time 0.001s
    p.BUF_OUTPUT(2, 40),           # step_bit = 40
    p.BUF_OUTPUT(3, 100),          # pre_n
    p.BUF_OUTPUT(4, 400),          # pulse_n
    p.BUF_OUTPUT(5, 500),          # post_n
    p.BUF_OUTPUT(7, 0x80808080),   # enable 1 and 2 motors on SG0

    p.BUF_OUTPUT(1, 2000),              # steps_val = 2000 (2 seconds)
    p.BUF_PARAM_ADDR(0),
    p.BUF_PARAM_WRITE_LO(0, 3),           # CH0.STATUS=3 (enable + target_v)
    p.BUF_PARAM_WRITE_LO(3, 0),           # CH0.V_OUT=0
    p.BUF_PARAM_WRITE_LO(1, 1500000),     # CH0.A
    p.BUF_PARAM_WRITE_LO(1, 0),           # CH0.J=0
    p.BUF_PARAM_WRITE_LO(1, 0),           # CH0.JJ=0
    p.BUF_PARAM_WRITE_LO(1, 2000000000),  # CH0.TARGET_V
    p.BUF_PARAM_WRITE_LO(1, 1500000),     # CH0.ABORT_A
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH1.STATUS=0
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH2
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH3
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH4
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH5
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH6
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH7
    p.BUF_STB(p.STB_ASG_START),   # Start ASG cycles
    
    p.BUF_WAIT_ALL(p.INT_ASG_LOAD),  # Wait for APG ready for new segment
    p.BUF_CLEAR(p.INT_ASG_LOAD),     # Clear pending int
    
    p.BUF_OUTPUT(1, 3000),           # steps_val = 3000 (3 seconds)
    p.BUF_PARAM_ADDR(0),
    p.BUF_PARAM_WRITE_LO(0, 3),          # CH0.STATUS=3
    p.BUF_PARAM_WRITE_LO(4, -1500000),   # CH0.A
    p.BUF_PARAM_WRITE_LO(3, 200000000),  # CH0.TARGET_V
    p.BUF_STB(p.STB_ASG_LOAD_DONE),  # Inform ASG, next segment setup is done

    p.BUF_WAIT_ALL(p.INT_ASG_LOAD),
    p.BUF_CLEAR(p.INT_ASG_LOAD),
    
    p.BUF_OUTPUT(1, 1000),      # steps_val = 1000 (1 second)
    p.BUF_PARAM_ADDR(0),
    p.BUF_PARAM_WRITE_LO(0, 3),          # CH0.STATUS=3
    p.BUF_PARAM_WRITE_LO(4, -1500000),   # CH0.A
    p.BUF_PARAM_WRITE_LO(3, 0),          # CH0.TARGET_V=0
    p.BUF_STB(p.STB_ASG_LOAD_DONE),
    
    p.BUF_WAIT_ALL(p.INT_ASG_LOAD),
    p.BUF_CLEAR(p.INT_ASG_LOAD),

    p.BUF_OUTPUT(1, 0),         # steps_val = 0 - End of path
    p.BUF_STB(p.STB_ASG_LOAD_DONE),
    p.BUF_WAIT_ALL(p.INT_ASG_DONE),  # Wait for ASG done
    p.BUF_CLEAR(p.INT_ASG_DONE),
    
    p.BUF_OUTPUT(7, 0x00000000), # disable 1 and 2 motors
    p.BUF_DONE(0)                # End of program for BE
]

p.S3G_STB(p.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_STB(p.STB_ASG_ABORT)
p.S3G_WRITE_FIFO(*a)           # Send program into FIFO
p.S3G_STB(p.STB_BE_START)      # Start execution

oa, ob, oc = None, None, None
while True:
    a = p.S3G_INPUT(p.IN_BE_STATUS)
    b = p.S3G_INPUT(p.IN_APG_STATUS)
    c = p.S3G_INPUT(p.IN_FIFO_DATA_COUNT)
    
    if (a,b,c) != (oa, ob, oc):
        print(a,b,c)
    (oa, ob, oc) = (a,b,c) 
    if a & 1 == 0:
        break
    time.sleep(0.01)

9 1 23
9 1 15
9 1 7
9 1 0
0 0 0
